# Munging the Titanic Dataset

## Recap of the need

During our exploration of the Titanic data, we found a few problems that need to be solved before the data is ready for analytics.

- About 31% (277 out of 891) of the values in `Age` are missing. Age seemed to play an important role, we need to estimate this in some way.
- While looking at the distributions, we saw that `Fare` seemed to contain extreme values at either end. A few tickets were probably provided free or there may be data entry errors. On the other hand "512" sounds like a very high fare for booking a ticket.
- We should also look at the non-numerical fields like `Ticket` and `Cabin` to see if they contain any useful information.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Note: 'head' is a UNIX command that works on a Mac but may not work on Windows.
# It lets us look at the head of a file that has not yet been read into Python.
# Don't worry if it doesn't work for you.
! head train.csv

In [ ]:
df = pd.read_csv("train.csv") 

Quick checks:

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.count()

### Check missing values

`Age` feature has just 714 values that aren't missing. `Cabin` is much worse. 

Let's work on `Cabin` first. A first glance at the variable leaves us with an impression that there are too many NaNs in the dataset. So, let us check the number of nulls / NaNs.

In [ ]:
sum(df['Cabin'].isnull())

`isnull()` returns 1, if the value is null. 687 is a lot of missing values. We'll drop `Cabin`.

Let's look at `Ticket`

In [ ]:
df.Ticket.unique()

`Ticket` has a mix of numbers and text and doesn’t seem to contain any information, so will drop `Ticket` as well.

In [ ]:
df = df.drop(['Ticket','Cabin'], axis=1)

In [ ]:
df.head()

### How should we handle missing values for `Age`?

There are a lot of possible ways to fill the missing values for `Age`.

One possibility is to drop any missing values. The downside is that we reduce the total number of rows for *all* variables.

In [ ]:
temp = df[pd.notnull(df['Age'])]

In [ ]:
temp.info()

The other extreme could be to build a supervised learning model to predict a`Age` on the basis of other variables and then use `Age` along with other variables to predict survival.

Another simple strategy is to fill the missing values for `Age` with the mean or median. That preserves all of the information for the other variables.

In [ ]:
meanAge = df.Age.mean()
temp = df.copy()
temp.Age = temp.Age.fillna(meanAge)
temp.info()

A better way to experiment with imputing missing data is to create a new column for each strategy.

In [ ]:
meanAge = df.Age.mean()
df['AgeFillMean'] = df.Age.fillna(meanAge)
df.head(10)

In [ ]:
medianAge = df.Age.median()
df['AgeFillMedian'] = df.Age.fillna(medianAge)
df.head(10)

### Simple model for missing values for `Age`

Suppose that we want to use the age that was typical in each passenger class and decide that the median might be better. Let's build another reference table to calculate each of these medians ...

In [ ]:
import numpy as np
median_ages = np.zeros((2,3))
median_ages

... and then populate the array.

First, we'll create a binary variable for Gender. We'll map 'female' to 0 and map 'male' to 1.

In [ ]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df.head()

In [ ]:
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & \
                              (df['Pclass'] == j+1)]['Age'].dropna().median()
 
median_ages

Again, we'll create a new column for this strategy for imputing missing values

In [ ]:
df['AgeFill'] = df['Age']
df.head(10)

Take a look at just the rows with missing values, and limit it to the columns important to us right now:

In [ ]:
df[df['Age'].isnull()][['Gender','Pclass','Age','AgeFill']].head(10)

Use some code to fill in `AgeFill` based on our median_ages table. Here we happen to use the alternate syntax for referring to an existing column, like `df.Age` rather than `df['Age']`. 

In [ ]:
for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]

Let's look at the same ten rows we just viewed.

In [ ]:
df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

This confirms we accomplished exactly what we wanted.

Let's also create a feature that records whether `Age` was originally missing. This is relatively simple by allowing pandas to use the integer conversion of the True/False evaluation of its built-in function, `pandas.isnull()`.

In [ ]:
df['AgeIsNull'] = pd.isnull(df.Age).astype(int)
df.head(10)

In [ ]:
df[['Survived', 'Pclass', 'Age', 'Fare', 'Gender', 'AgeFill', 'AgeIsNull']].describe()

Awesome! We now have a dataset that is almost ready for further analytics.

The next step is to drop the columns which we would not use:

In [ ]:
df = df.drop(['Name', 'Sex', 'Embarked'], axis=1)

We can also drop `Age`, since we copied and filled that to a better column `AgeFill`. The original `Age` still has the missing values which won't work well in our future model.

In [ ]:
df = df.drop(['Age'], axis=1)

An alternate way is to drop any rows which still have missing values:

In [ ]:
df = df.dropna()

In [ ]:
df.head(10)

### ... about those questionable values of `Fare` ...

In [ ]:
df.Fare.hist(bins = 50);

Let's replace the unusually high fare with the median.

In [ ]:
#ignore the warning here
df.Fare[df.Fare > 300] = df.Fare.median()

In [ ]:
df.Fare.hist(bins = 50);

The minimum fare is zero. Let's see if those might all be babies.

In [ ]:
df[['AgeFill','Fare']][df.Fare<5]

Notice that there is a jump from a fare of 0 to 4, so there is something going on here. Most likely, these are errors, so let’s replace them by the median fare for their class, and do the same for null values.

First we set those fares of 0 to NaN:

In [ ]:
# lambda just means a function we create on the fly
df.Fare = df.Fare.map(lambda x: np.nan if x==0 else x)

Calculate class medians

In [ ]:
class_medians = df.pivot_table('Fare', index='Pclass', aggfunc='median')
class_medians

Now try to write some code that replaces the NaN fare values with the class_medians.

In [ ]:
df.Fare.hist(bins = 50);

In [ ]:
df.describe()

## **Now** we have a clean and tidy dataset that is ready for analytics!